In [33]:
#  Functions to find all the Julia files in a directory and subdirectories
function filter_jl_files(dir_list::Vector{String})
    """Filter a list of filenames into a list of Julia files and a list of non-Julia files"""
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

function add_filepaths!(filepaths::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    """Add a list of filepaths to a dictionary"""
    for file in dir_list
        filepaths[file] = joinpath(dir_path, file)
    end
end

function find_jl_files(dir_path::String)
    """Find all Julia files in a directory and subdirectories"""
    filepaths = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(filepaths, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(filepaths, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return filepaths
end

find_jl_files (generic function with 1 method)

In [34]:
# Functions to read a Julia file, clean it, and return it as a long string
function clean_file(dir_path::String)
    """
        filestring = clean_file(dir_path::String)

    Read a Julia file line by line,
    removing all comments and doc strings,
    then appending each line to a string with a space
    """
    open(dir_path, "r") do file
        filestring = ""
        long_comment = false
        for line in eachline(file)
            # Check for doc strings and ignore them
            if contains(line, """\"\"\"""")
                long_comment = !long_comment
            end
            if long_comment
                continue
            else 
                # Remove comments
                if !startswith(strip(line), "#") && line != ""
                    # Remove tabs
                    line = replace(line, "\t" => " ")
                    # Make all one line, but add space to avoid concatenating words
                    filestring = string(filestring, " ", line)
                end
            end
        end
        return filestring
    end
end

function make_filestrings(filepath::Dict{String, String})
    """Make a dictionary of filestrings from a dictionary of filepaths"""
    filestrings = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestrings[filename] = clean_file(filepath)
    end
    return filestrings
end

make_filestrings (generic function with 1 method)

In [35]:
# Functions to find the names and creation-file of model features from a filestring
function extract_feature_name(filestring::AbstractString, filter_prefix::AbstractString)
    """Extract the name of a model feature from a file-substring"""
    x = split(filestring, ",")
    # Try to extract the name based on a prefix
    x2 = split(x[2], " ")
    x2 = filter(x -> startswith(x, filter_prefix), x2)
    if !isempty(x2)
        x = x2[1]
    else
        x = x[2]
    end
    # Remove any indexing
    if contains(x, "[")
        x = split(x, "[")[1]
    end
    # Remove whitespace
    return strip(x)
end

function find_feature_names(filestring::String, filter_string::String)
    """Create a list of the names of all the model feature of a given type, from a filestring"""
    cases = split(filestring, filter_string)
    feature_names = Vector{String}()
    filter_prefix = string(filter_string[2])
    for i in 2:length(cases)
        push!(feature_names, extract_feature_name(cases[i], filter_prefix))
    end
    return feature_names
end

function find_feature_creation!(feature_results::Dict{String, Any}, filestrings::Dict{String, String}, feature_name::String)
    """Find the files in which a set of features is created"""
    for (filename, filestring) in filestrings
        new_feature_names = find_feature_names(filestring, feature_name)
        for new_name in new_feature_names
            if !haskey(feature_results, new_name)
                feature_results[new_name] = Dict{String, Any}()
            end
            feature_results[new_name]["created"] = filename
        end
    end
end

find_feature_creation! (generic function with 1 method)

In [36]:
# Functions to find where each model feature is accessed
function check_access(filestrings::Dict{String, String}, feature_names::Vector{String})
    """Check if features are accessed in a set of files, using filestrings"""
    file_accesses = Dict{String, Vector{String}}()
    for (filename, filestring) in filestrings
        file_accesses[filename] = Vector{String}()
        for feature_name in feature_names
            if contains(filestring, feature_name)
                push!(file_accesses[filename], feature_name)
            end
        end
    end
    return file_accesses
end

function push_accesses!(feature_results::Dict{String, Any}, accesses::Dict{String, Vector{String}})
    """Push a list of features accesses to a feature result dictionary"""
    for (filename, feature_names) in accesses
        for feature_name in feature_names
            if !haskey(feature_results[feature_name], "accessed")
                feature_results[feature_name]["accessed"] = Vector{String}()
            end
            push!(feature_results[feature_name]["accessed"], filename)
        end
    end
end

function find_feature_accesses!(feature_results::Dict{String, Any}, filestrings::Dict{String, String})
    """Find the files in which a set of features is accessed"""
    accesses = check_access(filestrings, collect(keys(feature_results)))
    push_accesses!(feature_results, accesses)
end

find_feature_accesses! (generic function with 1 method)

In [37]:
# Functions to turn search_results into a Markdown table
function make_table_header(headers::Set{String}, feature_type::String)
    fun_headers = copy(headers)
    table_header = "|$(feature_type) name"
    table_divider = "|:-"
    if "created" in fun_headers # Later we can automate this using the features_to_search superdetails
        table_header = string(table_header, "|created")
        table_divider = string(table_divider, "|:-")
        delete!(fun_headers, "created")
    end    
    for header in fun_headers
        table_header = string(table_header, "|", header)
        table_divider = string(table_divider, "|", ":-")
    end
    table_header = string(table_header, "|")
    table_divider = string(table_divider, "|")
    return table_header, table_divider
end

function make_table_row(feature_name::String, feature_results::Dict{String, Any}, headers::Set{String})
    table_row = "|$(feature_name)"
    fun_headers = copy(headers)
    if "created" in fun_headers
        table_row = string(table_row, "|", feature_results["created"])
        delete!(fun_headers, "created")
    end
    for header in fun_headers
        table_row = string(table_row, "|", join(feature_results[header], ", "))
    end
    table_row = string(table_row, "|")
    return table_row
end

function data_2_markdown(feature_results::Dict{String, Any}, feature_type::String, headers::Set{String})
    md_table = Vector{String}()
    push!(md_table, "## $(feature_type)s")
    table_header, table_divider = make_table_header(headers, feature_type)
    push!(md_table, table_header)
    push!(md_table, table_divider)
    sorted_names = sort(collect(keys(feature_results)))
    for name in sorted_names
        push!(md_table, make_table_row(name, feature_results[name], headers))
    end
    return md_table
end

function write_2_file(filename::String, md_table::Vector{String})
    open(filename, "w") do f
        for line in md_table
            write(f, string(line, "\n"))
        end
    end
end

function write_2_file(filename::String, md_table::Vector{String}, open_mode::String)
    open(filename, open_mode) do f
        for line in md_table
            write(f, string(line, "\n"))
        end
    end
end

write_2_file (generic function with 2 methods)

In [38]:
# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
filepaths = find_jl_files(src_path)

outpath = joinpath(pwd(), "model_table.md")
open(outpath, "w") do io
    println(io, "# Model Variables and Expressions")
end

# Clean all Julia files and return them as strings
filestrings = make_filestrings(filepaths)

# Create a dictionary of all the model features we'd like to search for, and the details of interest
features_to_search = Dict(
    "var" => Dict("filter_string" => "@variable", "table_name" => "Variable", "details" => ["created", "accessed"], "superdetails" => ["created"]),
    "exp" => Dict("filter_string" => "@expression", "table_name" => "Expression", "details" => ["created", "accessed"], "superdetails" => ["created"])
)

# Create dictionary to hold search results
# We could be more specific and make the last Dict{String, String||List}, but this is safer
search_results = Dict{String, Dict{String, Any}}()
for (search_name, search_details) in features_to_search
    search_results[search_name] = Dict{String, Dict{String, Any}}()
end

# Our dict of results is not ordered, so we specify that here
print_order = Vector{String}(["var", "exp"])

# Find the file in which each feature is created
for search_name in print_order
    search_details = features_to_search[search_name]
    find_feature_creation!(search_results[search_name], filestrings, search_details["filter_string"])
    find_feature_accesses!(search_results[search_name], filestrings)
    md_table = data_2_markdown(search_results[search_name], search_details["table_name"], Set(search_details["details"]))
    write_2_file(outpath, md_table, open_mode)
end

In [39]:
filename = "thermal_commit.jl"
filepath = filepaths[filename]
filestring = clean_file(filepath)
q = split(filestring, "function ")
q = q[2:end]
for w in q
    # w = q[1]
    w = split(w, "(")
    fun_name = w[1]
    fun_args = split(split(w[2], ")")[1], ", ")
    println(fun_name)
    for a in fun_args
        println(a)
    end
end

thermal_commit
EP::Model
inputs::Dict
Reserves::Int
thermal_commit_reserves
EP::Model
inputs::Dict
hoursbefore
p::Int
t::Int
b::UnitRange{Int}


In [40]:
function find_functions(filepaths::Dict{String, String})
    function_names = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestring = clean_file(filepath)
        new_functions = filter_filestring(filestring, "function ")
        for fun in new_functions
            if !haskey(function_names, fun)
                function_names[fun] = filename
            end
        end
    end
    return function_names
end

find_functions (generic function with 1 method)